# Compiling Circuits for DM4 and DL5
The following notebook cells create circuit graphs to be executed. After compiling our circuits here, we will simulate them in the CircuitSimulation notebook.
## Setup
For circuit generation, use the (optional) install below. You will also require a working Neurokernel/Neurodriver installation for the CircuitSimulation notebook. Following the FlyBrainLab Full Installation instructions at https://github.com/FlyBrainLab/FlyBrainLab#12-full-installation or the Docker image for FlyBrainLab might be useful for setting up your environment.

In [ ]:
# This is Optional!
# !pip install scikit-learn
# !pip install networkx

# Import Circuit Generation Utilities
The code below imports a number of functions from the library manually for future compatibility. You can replace it with 

``
from subpackages.antennallobe.antennal_lobe_circuits import *
``

in your own code.

In [ ]:
import pickle
import numpy as np
import networkx as nx
from sklearn.metrics import pairwise_distances

def read_pickle(x):
    with (open(x, "rb")) as f:
        data = pickle.load(f)
    return data

X = np.load('al_synapses2.npy')
n_data = np.load('al_references.npy')
n_data_dict = {}
for i in range(n_data.shape[0]):
    n_data_dict[n_data[i,1]] = n_data[i,0]

ORN_PN_pos = read_pickle('ORN_PN_pos.pickle')
ORN_LN_pos = read_pickle('ORN_LN_pos.pickle')
LN_PN_pos = read_pickle('LN_PN_pos.pickle')
PN_LN_pos = read_pickle('PN_LN_pos.pickle')
LN_ORN_pos = read_pickle('LN_ORN_pos.pickle')
LN_LN_pos = read_pickle('LN_LN_pos.pickle')

uniques = list(np.unique(X[:,3]))
print('uniques',len(uniques))
ORN_uniques = [i for i in uniques if 'ORN' in i]
PN_uniques = [i for i in uniques if 'PN' in i]
LN_uniques = [i for i in uniques if 'LN' in i]
print('ORN uniques',len(ORN_uniques))
print('PN uniques',len(PN_uniques))
print('LN uniques',len(LN_uniques))

def get_LNs(glom = 'DL5'):
    found_LNs = []
    for i in [i for i in ORN_uniques if glom in i]:
        for j in LN_uniques:
            if j in ORN_LN_pos[i]:
                found_LNs.append(j)
    for i in [i for i in PN_uniques if glom in i]:
        for j in LN_uniques:
            if j in PN_LN_pos[i]:
                found_LNs.append(j)
    for j in LN_uniques:
        for i in [i for i in ORN_uniques if glom in i]:
            if i in LN_ORN_pos[j]:
                found_LNs.append(j)
    for j in LN_uniques:
        for i in [i for i in PN_uniques if glom in i]:
            if i in LN_PN_pos[j]:
                found_LNs.append(j)
    found_LNs = list(set(found_LNs))
    return found_LNs
                
DL5_LNs = get_LNs('DL5')
DC1_LNs = get_LNs('DC1')

only_DL5_LNs = [i for i in DL5_LNs if i not in DC1_LNs]
only_DC1_LNs = [i for i in DC1_LNs if i not in DL5_LNs]
remnant_LNs = [i for i in LN_uniques if i not in only_DL5_LNs and i not in only_DC1_LNs]

In [ ]:
def setup_spiking_default(ORN_PN_gain = 1., ORN_LN_gain = 1., LN_PN_gain = 1., PN_LN_gain = 1., interaction_gain = 1., LN_LN_gain=1., exLN_LN_gain=1., LN_exLN_gain=1.):
    # default circuit setting; note that this is optimized for when the no. of inputs is double 
    def gen_ORN(G, x):
        params = dict(
            br=1.0,
            dr=10.0,
            gamma=0.138,
            a1=45.0,
            b1=0.8,
            a2=199.574,
            b2=51.887,
            a3=2.539,
            b3=0.9096,
            kappa=9593.9,
            p=1.0,
            c=0.06546,
            Imax=150.159,
        )
        G.add_node(x+'_OTP', **{"class": "OTP"}, **params)

        params = dict(
            ms=-5.3,
            ns=-4.3,
            hs=-12.0,
            gNa=120.0,
            gK=20.0,
            gL=0.3,
            ga=47.7,
            ENa=55.0,
            EK=-72.0,
            EL=-17.0,
            Ea=-75.0,
            sigma=0.00,
            refperiod=0.0,
        )
        G.add_node(x+'_RN', **{"class": "NoisyConnorStevens"}, **params)
        G.add_edge(x+'_OTP', x+'_RN')

        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=0.6/10000.*ORN_PN_gain,
        )
        G.add_node(x+'_ARN', **{"class": "Alpha"}, **params)
        G.add_edge(x+'_RN', x+'_ARN')

        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=0.6/1000.*ORN_LN_gain,
        )
        G.add_node(x+'_ARN_LN', **{"class": "Alpha"}, **params)
        G.add_edge(x+'_RN', x+'_ARN_LN')

    def gen_PN(G, x):
        params = dict(
            ms=-5.3,
            ns=-4.3,
            hs=-12.0,
            gNa=120.0,
            gK=20.0,
            gL=0.3,
            ga=47.7,
            ENa=55.0,
            EK=-72.0,
            EL=-17.0,
            Ea=-75.0,
            sigma=0.00,
            refperiod=0.0,
        )
        G.add_node(x+'_PN', **{"class": "NoisyConnorStevens"}, **params)

    def gen_LN(G, x):
        params = dict(
            ms=-5.3,
            ns=-4.3,
            hs=-12.0,
            gNa=120.0,
            gK=20.0,
            gL=0.3,
            ga=47.7,
            ENa=55.0,
            EK=-72.0,
            EL=-17.0,
            Ea=-75.0,
            sigma=0.00,
            refperiod=0.0,
        )
        G.add_node(x, **{"class": "NoisyConnorStevens"}, **params)

    def ORN_PN_LN_ORN_interaction(G,x,y,z,i,j, dist_gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./100. * dist_gain * interaction_gain,
        )
        G.add_node(x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), **{"class": "Alpha"}, **params)
        G.add_edge(x, x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j))

        params = dict(
            dummy=0.0,
        )
        G.add_node(x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j), **{"class": "PreLN"}, **params) # LN>(LN>ORN)
        G.add_edge(x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j))
        #G.add_node(x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j), **{"class": "OSNAxt2"}, **params) # ORN>(ORN>PN)
        #G.add_edge(y+'_ARN', x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j))
        #G.add_edge(x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j), z)
        G.add_edge(x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j), y+'_'+z+'_AT'+'_'+str(j)) # (LN>ORN) to (ORN>PN)
        return [x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), 'g']

    def ORN_ORN_LN_interaction(G,x,y,z,i,j, dist_gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./100. * dist_gain * interaction_gain,
        )
        G.add_node(x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), **{"class": "Alpha"}, **params)
        G.add_edge(x, x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j))

        params = dict(
            dummy=0.0,
        )
        G.add_node(x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j), **{"class": "PreLN"}, **params) # LN>(LN>ORN)
        G.add_edge(x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j))
        #G.add_node(x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j), **{"class": "OSNAxt2"}, **params) # ORN>(ORN>PN)
        #G.add_edge(y+'_ARN', x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j))
        #G.add_edge(x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j), z)
        G.add_edge(x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j), y+'_'+z+'_AT'+'_'+str(j)) # (LN>ORN) to (ORN>PN)
        return [y+'_'+z+'_AT'+'_'+str(j), 'g']

    def gen_ORNPN_syn(G, x, y, i, gain=1.):
        params = dict(
            bias=1.0,
            gain=1. * gain,
        )
        G.add_node(x+'_'+y+'_AT'+'_'+str(i), **{"class": "OSNAxt2"}, **params) # ORN>(ORN>PN)
        G.add_edge(x+'_ARN', x+'_'+y+'_AT'+'_'+str(i))
        G.add_edge(x+'_'+y+'_AT'+'_'+str(i), y+'_PN')
        return [x+'_'+y+'_AT'+'_'+str(i), 'I']

    def gen_ORNLN_syn(G, x, y, i, gain=1.):
        params = dict(
            bias=1.0,
            gain=1. * gain,
        )
        G.add_node(x+'_'+y+'_AT'+'_'+str(i), **{"class": "OSNAxt2"}, **params) # ORN>(ORN>PN)
        G.add_edge(x+'_ARN_LN', x+'_'+y+'_AT'+'_'+str(i))
        G.add_edge(x+'_'+y+'_AT'+'_'+str(i), y)
        return [x+'_'+y+'_AT'+'_'+str(i), 'I']

    def gen_regsyn(G, x, y, i, gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./10000.*gain*PN_LN_gain,
        )
        G.add_node(x+'_to_'+y+'_Alpha_'+str(i), **{"class": "Alpha"}, **params)
        params = dict(
            bias=1.0,
            gain=1.,
        )
        G.add_node(x+'_to_'+y+'_Converter_'+str(i), **{"class": "OSNAxt2"}, **params)
        G.add_edge(x+'_PN', x+'_to_'+y+'_Alpha_'+str(i))
        G.add_edge(x+'_to_'+y+'_Alpha_'+str(i), x+'_to_'+y+'_Converter_'+str(i))
        G.add_edge(x+'_to_'+y+'_Converter_'+str(i), y)
        return [x+'_to_'+y+'_Alpha_'+str(i), 'g']

    def gen_regsyn_PN(G, x, y, i, gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./10000.*gain*np.abs(LN_PN_gain),
        )
        G.add_node(x+'_to_'+y+'_Alpha_'+str(i), **{"class": "Alpha"}, **params)
        params = dict(
            bias=1.0,
            gain=1.*np.sign(LN_PN_gain),
        )
        G.add_node(x+'_to_'+y+'_Converter_'+str(i), **{"class": "OSNAxt2"}, **params)
        G.add_edge(x, x+'_to_'+y+'_Alpha_'+str(i))
        G.add_edge(x+'_to_'+y+'_Alpha_'+str(i), x+'_to_'+y+'_Converter_'+str(i))
        G.add_edge(x+'_to_'+y+'_Converter_'+str(i), y+'_PN')
        return [x+'_to_'+y+'_Alpha_'+str(i), 'g']

    def gen_regsyn_LN(G, x, y, i, gain=1.):

        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./1000.*gain,
        )

        G.add_node(x+'_to_'+y+'_Alpha_'+str(i), **{"class": "Alpha"}, **params)

        params = dict(
            bias=1.0,
            gain=1.,
        )

        G.add_node(x+'_to_'+y+'_Converter_'+str(i), **{"class": "OSNAxt2"}, **params)
        G.add_edge(x+'_RN', x+'_to_'+y+'_Alpha_'+str(i))
        G.add_edge(x+'_to_'+y+'_Alpha_'+str(i), x+'_to_'+y+'_Converter_'+str(i))
        G.add_edge(x+'_to_'+y+'_Converter_'+str(i), y)
        return [x+'_to_'+y+'_Alpha_'+str(i), 'g']
    
    def gen_regsyn_LN2(G, x, y, i, gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./10000.*gain*LN_LN_gain,
        )
        G.add_node(x+'_to_'+y+'_Alpha_'+str(i), **{"class": "Alpha"}, **params)
        params = dict(
            bias=1.0,
            gain=-1.,
        )
        G.add_node(x+'_to_'+y+'_Converter_'+str(i), **{"class": "OSNAxt2"}, **params)
        G.add_edge(x, x+'_to_'+y+'_Alpha_'+str(i))
        G.add_edge(x+'_to_'+y+'_Alpha_'+str(i), x+'_to_'+y+'_Converter_'+str(i))
        G.add_edge(x+'_to_'+y+'_Converter_'+str(i), y)
        return [x+'_to_'+y+'_Alpha_'+str(i), 'g']
    
    def gen_regsyn_LNex(G, x, y, i, gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./1000000.*gain*exLN_LN_gain,
        )
        G.add_node(x+'_to_'+y+'_Alpha_'+str(i), **{"class": "Alpha"}, **params)
        params = dict(
            bias=1.0,
            gain=1.,
        )
        G.add_node(x+'_to_'+y+'_Converter_'+str(i), **{"class": "OSNAxt2"}, **params)
        G.add_edge(x, x+'_to_'+y+'_Alpha_'+str(i))
        G.add_edge(x+'_to_'+y+'_Alpha_'+str(i), x+'_to_'+y+'_Converter_'+str(i))
        G.add_edge(x+'_to_'+y+'_Converter_'+str(i), y)
        return [x+'_to_'+y+'_Alpha_'+str(i), 'g']
    
    def gen_regsyn_exLN(G, x, y, i, gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./1000000.*gain*LN_exLN_gain,
        )
        G.add_node(x+'_to_'+y+'_Alpha_'+str(i), **{"class": "Alpha"}, **params)
        params = dict(
            bias=1.0,
            gain=-1.,
        )
        G.add_node(x+'_to_'+y+'_Converter_'+str(i), **{"class": "OSNAxt2"}, **params)
        G.add_edge(x, x+'_to_'+y+'_Alpha_'+str(i))
        G.add_edge(x+'_to_'+y+'_Alpha_'+str(i), x+'_to_'+y+'_Converter_'+str(i))
        G.add_edge(x+'_to_'+y+'_Converter_'+str(i), y)
        return [x+'_to_'+y+'_Alpha_'+str(i), 'g']

    neuron_models = {'ORNs': gen_ORN, 'PNs': gen_PN, 'LNs': gen_LN, 'exLNs': gen_LN}
    synapse_models = {'ORNs-LNs': gen_ORNLN_syn, 'LNs-PNs': gen_regsyn_PN, 'PNs-LNs': gen_regsyn, 'ORNs-PNs': gen_ORNPN_syn, 'LNs-LNs': gen_regsyn_LN2, 'LNs-exLNs': gen_regsyn_LNex, 'exLNs-LNs': gen_regsyn_exLN}
    interaction_models = {'LNs-ORNs-PNs': ORN_PN_LN_ORN_interaction, 'LNs-ORNs-LNs': ORN_PN_LN_ORN_interaction}
    return neuron_models, synapse_models, interaction_models

In [ ]:
def setup_spiking_default_withLNs(ORN_PN_gain = 1., ORN_LN_gain = 1., LN_PN_gain = 1.):
    def gen_ORN(G, x):
        params = dict(
            br=1.0,
            dr=10.0,
            gamma=0.138,
            a1=45.0,
            b1=0.8,
            a2=199.574,
            b2=51.887,
            a3=2.539,
            b3=0.9096,
            kappa=9593.9,
            p=1.0,
            c=0.06546,
            Imax=150.159,
        )
        G.add_node(x+'_OTP', **{"class": "OTP"}, **params)

        params = dict(
            ms=-5.3,
            ns=-4.3,
            hs=-12.0,
            gNa=120.0,
            gK=20.0,
            gL=0.3,
            ga=47.7,
            ENa=55.0,
            EK=-72.0,
            EL=-17.0,
            Ea=-75.0,
            sigma=0.00,
            refperiod=0.0,
        )
        G.add_node(x+'_RN', **{"class": "NoisyConnorStevens"}, **params)
        G.add_edge(x+'_OTP', x+'_RN')

        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=0.6/10000.*ORN_PN_gain,
        )
        G.add_node(x+'_ARN', **{"class": "Alpha"}, **params)
        G.add_edge(x+'_RN', x+'_ARN')

        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=0.6/1000.*ORN_LN_gain,
        )
        G.add_node(x+'_ARN_LN', **{"class": "Alpha"}, **params)
        G.add_edge(x+'_RN', x+'_ARN_LN')

    def gen_PN(G, x):
        params = dict(
            ms=-5.3,
            ns=-4.3,
            hs=-12.0,
            gNa=120.0,
            gK=20.0,
            gL=0.3,
            ga=47.7,
            ENa=55.0,
            EK=-72.0,
            EL=-17.0,
            Ea=-75.0,
            sigma=0.00,
            refperiod=0.0,
        )
        G.add_node(x+'_PN', **{"class": "NoisyConnorStevens"}, **params)

    def gen_LN(G, x):
        params = dict(
            ms=-5.3,
            ns=-4.3,
            hs=-12.0,
            gNa=120.0,
            gK=20.0,
            gL=0.3,
            ga=47.7,
            ENa=55.0,
            EK=-72.0,
            EL=-17.0,
            Ea=-75.0,
            sigma=0.00,
            refperiod=0.0,
        )
        G.add_node(x, **{"class": "NoisyConnorStevens"}, **params)

    def ORN_PN_LN_ORN_interaction(G,x,y,z,i,j, dist_gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./100. * dist_gain,
        )
        G.add_node(x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), **{"class": "Alpha"}, **params)
        G.add_edge(x, x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j))

        params = dict(
            dummy=0.0,
        )
        G.add_node(x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j), **{"class": "PreLN"}, **params) # LN>(LN>ORN)
        G.add_edge(x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j))
        #G.add_node(x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j), **{"class": "OSNAxt2"}, **params) # ORN>(ORN>PN)
        #G.add_edge(y+'_ARN', x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j))
        #G.add_edge(x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j), z)
        G.add_edge(x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j), y+'_'+z+'_AT'+'_'+str(j)) # (LN>ORN) to (ORN>PN)
        return [x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), 'g']

    def ORN_ORN_LN_interaction(G,x,y,z,i,j, dist_gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./100. * dist_gain,
        )
        G.add_node(x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), **{"class": "Alpha"}, **params)
        G.add_edge(x, x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j))

        params = dict(
            dummy=0.0,
        )
        G.add_node(x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j), **{"class": "PreLN"}, **params) # LN>(LN>ORN)
        G.add_edge(x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j))
        #G.add_node(x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j), **{"class": "OSNAxt2"}, **params) # ORN>(ORN>PN)
        #G.add_edge(y+'_ARN', x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j))
        #G.add_edge(x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j), z)
        G.add_edge(x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j), y+'_'+z+'_AT'+'_'+str(j)) # (LN>ORN) to (ORN>PN)
        return [y+'_'+z+'_AT'+'_'+str(j), 'g']

    def gen_ORNPN_syn(G, x, y, i, gain=1.):
        params = dict(
            bias=1.0,
            gain=1.0,
        )
        G.add_node(x+'_'+y+'_AT'+'_'+str(i), **{"class": "OSNAxt2"}, **params) # ORN>(ORN>PN)
        G.add_edge(x+'_ARN', x+'_'+y+'_AT'+'_'+str(i))
        G.add_edge(x+'_'+y+'_AT'+'_'+str(i), y+'_PN')
        return [x+'_'+y+'_AT'+'_'+str(i), 'I']

    def gen_ORNLN_syn(G, x, y, i, gain=1.):
        params = dict(
            bias=1.0,
            gain=1.0,
        )
        G.add_node(x+'_'+y+'_AT'+'_'+str(i), **{"class": "OSNAxt2"}, **params) # ORN>(ORN>PN)
        G.add_edge(x+'_ARN_LN', x+'_'+y+'_AT'+'_'+str(i))
        G.add_edge(x+'_'+y+'_AT'+'_'+str(i), y)
        return [x+'_'+y+'_AT'+'_'+str(i), 'I']

    def gen_regsyn(G, x, y, i, gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./10000.*gain,
        )
        G.add_node(x+'_to_'+y+'_Alpha_'+str(i), **{"class": "Alpha"}, **params)
        params = dict(
            bias=1.0,
            gain=1.,
        )
        G.add_node(x+'_to_'+y+'_Converter_'+str(i), **{"class": "OSNAxt2"}, **params)
        G.add_edge(x+'_PN', x+'_to_'+y+'_Alpha_'+str(i))
        G.add_edge(x+'_to_'+y+'_Alpha_'+str(i), x+'_to_'+y+'_Converter_'+str(i))
        G.add_edge(x+'_to_'+y+'_Converter_'+str(i), y)
        return [x+'_to_'+y+'_Alpha_'+str(i), 'g']

    def gen_regsyn_PN(G, x, y, i, gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./10000.*gain*LN_PN_gain,
        )
        G.add_node(x+'_to_'+y+'_Alpha_'+str(i), **{"class": "Alpha"}, **params)
        params = dict(
            bias=1.0,
            gain=1.,
        )
        G.add_node(x+'_to_'+y+'_Converter_'+str(i), **{"class": "OSNAxt2"}, **params)
        G.add_edge(x, x+'_to_'+y+'_Alpha_'+str(i))
        G.add_edge(x+'_to_'+y+'_Alpha_'+str(i), x+'_to_'+y+'_Converter_'+str(i))
        G.add_edge(x+'_to_'+y+'_Converter_'+str(i), y+'_PN')
        return [x+'_to_'+y+'_Alpha_'+str(i), 'g']

    def gen_regsyn_LN(G, x, y, i, gain=1.):

        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./1000.*gain,
        )

        G.add_node(x+'_to_'+y+'_Alpha_'+str(i), **{"class": "Alpha"}, **params)

        params = dict(
            bias=1.0,
            gain=1.,
        )

        G.add_node(x+'_to_'+y+'_Converter_'+str(i), **{"class": "OSNAxt2"}, **params)
        G.add_edge(x+'_RN', x+'_to_'+y+'_Alpha_'+str(i))
        G.add_edge(x+'_to_'+y+'_Alpha_'+str(i), x+'_to_'+y+'_Converter_'+str(i))
        G.add_edge(x+'_to_'+y+'_Converter_'+str(i), y)
        return [x+'_to_'+y+'_Alpha_'+str(i), 'g']

    def gen_regsyn_LNLN(G, x, y, i, gain=1.):

        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./1000.*gain,
        )

        G.add_node(x+'_to_'+y+'_Alpha_'+str(i), **{"class": "Alpha"}, **params)

        params = dict(
            bias=1.0,
            gain=1.,
        )

        G.add_node(x+'_to_'+y+'_Converter_'+str(i), **{"class": "OSNAxt2"}, **params)
        G.add_edge(x, x+'_to_'+y+'_Alpha_'+str(i))
        G.add_edge(x+'_to_'+y+'_Alpha_'+str(i), x+'_to_'+y+'_Converter_'+str(i))
        G.add_edge(x+'_to_'+y+'_Converter_'+str(i), y)
        return [x+'_to_'+y+'_Alpha_'+str(i), 'g']

    
    neuron_models = {'ORNs': gen_ORN, 'PNs': gen_PN, 'LNs': gen_LN}
    synapse_models = {'ORNs-LNs': gen_ORNLN_syn, 'LNs-PNs': gen_regsyn_PN, 'LNs-LNs': gen_regsyn_LNLN, 'PNs-LNs': gen_regsyn, 'ORNs-PNs': gen_ORNPN_syn}
    interaction_models = {'LNs-ORNs-PNs': ORN_PN_LN_ORN_interaction, 'LNs-ORNs-LNs': ORN_PN_LN_ORN_interaction}
    return neuron_models, synapse_models, interaction_models

In [ ]:
def setup_spiking_simplest(ORN_PN_gain = 1., ORN_LN_gain = 1., LN_PN_gain = 1., LN_LN_gain=1.):
    def gen_ORN(G, x):
        params = dict(
            br=1.0,
            dr=10.0,
            gamma=0.138,
            a1=45.0,
            b1=0.8,
            a2=199.574,
            b2=51.887,
            a3=2.539,
            b3=0.9096,
            kappa=9593.9,
            p=1.0,
            c=0.06546,
            Imax=150.159,
        )
        G.add_node(x+'_OTP', **{"class": "OTP"}, **params)

        params = dict(
            ms=-5.3,
            ns=-4.3,
            hs=-12.0,
            gNa=120.0,
            gK=20.0,
            gL=0.3,
            ga=47.7,
            ENa=55.0,
            EK=-72.0,
            EL=-17.0,
            Ea=-75.0,
            sigma=0.00,
            refperiod=0.0,
        )
        G.add_node(x+'_RN', **{"class": "NoisyConnorStevens"}, **params)
        G.add_edge(x+'_OTP', x+'_RN')

        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=0.9/10000.*ORN_PN_gain,
        )
        G.add_node(x+'_ARN', **{"class": "Alpha"}, **params)
        G.add_edge(x+'_RN', x+'_ARN')

        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=0.3/1000.*ORN_LN_gain,
        )
        G.add_node(x+'_ARN_LN', **{"class": "Alpha"}, **params)
        G.add_edge(x+'_RN', x+'_ARN_LN')

    def gen_PN(G, x):
        params = dict(
            ms=-5.3,
            ns=-4.3,
            hs=-12.0,
            gNa=120.0,
            gK=20.0,
            gL=0.3,
            ga=47.7,
            ENa=55.0,
            EK=-72.0,
            EL=-17.0,
            Ea=-75.0,
            sigma=0.00,
            refperiod=10.0,
        )
        G.add_node(x+'_PN', **{"class": "NoisyConnorStevens"}, **params)

    def gen_LN(G, x):
        params = dict(
            ms=-5.3,
            ns=-4.3,
            hs=-12.0,
            gNa=120.0,
            gK=20.0,
            gL=0.3,
            ga=47.7,
            ENa=55.0,
            EK=-72.0,
            EL=-17.0,
            Ea=-75.0,
            sigma=0.00,
            refperiod=0.0,
        )
        G.add_node(x, **{"class": "NoisyConnorStevens"}, **params)

    def ORN_PN_LN_ORN_interaction(G,x,y,z,i,j, dist_gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./100. * dist_gain,
        )
        G.add_node(x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), **{"class": "Alpha"}, **params)
        G.add_edge(x, x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j))

        params = dict(
            dummy=0.0,
        )
        G.add_node(x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j), **{"class": "PreLN"}, **params) # LN>(LN>ORN)
        G.add_edge(x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j))
        #G.add_node(x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j), **{"class": "OSNAxt2"}, **params) # ORN>(ORN>PN)
        #G.add_edge(y+'_ARN', x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j))
        #G.add_edge(x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j), z)
        G.add_edge(x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j), y+'_'+z+'_AT'+'_'+str(j)) # (LN>ORN) to (ORN>PN)
        return [x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), 'g']

    def ORN_ORN_LN_interaction(G,x,y,z,i,j, dist_gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./100. * dist_gain,
        )
        G.add_node(x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), **{"class": "Alpha"}, **params)
        G.add_edge(x, x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j))

        params = dict(
            dummy=0.0,
        )
        G.add_node(x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j), **{"class": "PreLN"}, **params) # LN>(LN>ORN)
        G.add_edge(x+'_to_'+y+'_PreLNAlpha'+'_'+str(z)+'_'+str(i)+'_'+str(j), x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j))
        #G.add_node(x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j), **{"class": "OSNAxt2"}, **params) # ORN>(ORN>PN)
        #G.add_edge(y+'_ARN', x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j))
        #G.add_edge(x+'_'+y+'_to_'+z+'_AT'+'_'+str(i)+'_'+str(j), z)
        G.add_edge(x+'_to_'+y+'_PreLN'+'_'+str(z)+'_'+str(i)+'_'+str(j), y+'_'+z+'_AT'+'_'+str(j)) # (LN>ORN) to (ORN>PN)
        return [y+'_'+z+'_AT'+'_'+str(j), 'g']

    def gen_ORNPN_syn(G, x, y, i, gain=1.):
        params = dict(
            bias=1.0,
            gain=1.0,
        )
        G.add_node(x+'_'+y+'_AT'+'_'+str(i), **{"class": "OSNAxt2"}, **params) # ORN>(ORN>PN)
        G.add_edge(x+'_ARN', x+'_'+y+'_AT'+'_'+str(i))
        G.add_edge(x+'_'+y+'_AT'+'_'+str(i), y+'_PN')
        return [x+'_'+y+'_AT'+'_'+str(i), 'I']

    def gen_ORNLN_syn(G, x, y, i, gain=1.):
        params = dict(
            bias=1.0,
            gain=1.0,
        )
        G.add_node(x+'_'+y+'_AT'+'_'+str(i), **{"class": "OSNAxt2"}, **params) # ORN>(ORN>PN)
        G.add_edge(x+'_ARN_LN', x+'_'+y+'_AT'+'_'+str(i))
        G.add_edge(x+'_'+y+'_AT'+'_'+str(i), y)
        return [x+'_'+y+'_AT'+'_'+str(i), 'I']

    def gen_regsyn(G, x, y, i, gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./10000.*gain,
        )
        G.add_node(x+'_to_'+y+'_Alpha_'+str(i), **{"class": "Alpha"}, **params)
        params = dict(
            bias=1.0,
            gain=1.,
        )
        G.add_node(x+'_to_'+y+'_Converter_'+str(i), **{"class": "OSNAxt2"}, **params)
        G.add_edge(x+'_PN', x+'_to_'+y+'_Alpha_'+str(i))
        G.add_edge(x+'_to_'+y+'_Alpha_'+str(i), x+'_to_'+y+'_Converter_'+str(i))
        G.add_edge(x+'_to_'+y+'_Converter_'+str(i), y)
        return [x+'_to_'+y+'_Alpha_'+str(i), 'g']

    def gen_regsyn_PN(G, x, y, i, gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./10000.*gain*LN_PN_gain,
        )
        G.add_node(x+'_to_'+y+'_Alpha_'+str(i), **{"class": "Alpha"}, **params)
        params = dict(
            bias=1.0,
            gain=1.,
        )
        G.add_node(x+'_to_'+y+'_Converter_'+str(i), **{"class": "OSNAxt2"}, **params)
        G.add_edge(x, x+'_to_'+y+'_Alpha_'+str(i))
        G.add_edge(x+'_to_'+y+'_Alpha_'+str(i), x+'_to_'+y+'_Converter_'+str(i))
        G.add_edge(x+'_to_'+y+'_Converter_'+str(i), y+'_PN')
        return [x+'_to_'+y+'_Alpha_'+str(i), 'g']
    
    def gen_regsyn_LN2(G, x, y, i, gain=1.):
        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=-1./10000.*gain*LN_LN_gain,
        )
        G.add_node(x+'_to_'+y+'_Alpha_'+str(i), **{"class": "Alpha"}, **params)
        params = dict(
            bias=1.0,
            gain=1.,
        )
        G.add_node(x+'_to_'+y+'_Converter_'+str(i), **{"class": "OSNAxt2"}, **params)
        G.add_edge(x, x+'_to_'+y+'_Alpha_'+str(i))
        G.add_edge(x+'_to_'+y+'_Alpha_'+str(i), x+'_to_'+y+'_Converter_'+str(i))
        G.add_edge(x+'_to_'+y+'_Converter_'+str(i), y+'_PN')
        return [x+'_to_'+y+'_Alpha_'+str(i), 'g']

    def gen_regsyn_LN(G, x, y, i, gain=1.):

        params = dict(
            ar=12.5,
            ad=12.19,
            gmax=1./1000.*gain,
        )

        G.add_node(x+'_to_'+y+'_Alpha_'+str(i), **{"class": "Alpha"}, **params)

        params = dict(
            bias=1.0,
            gain=1.,
        )

        G.add_node(x+'_to_'+y+'_Converter_'+str(i), **{"class": "OSNAxt2"}, **params)
        G.add_edge(x+'_RN', x+'_to_'+y+'_Alpha_'+str(i))
        G.add_edge(x+'_to_'+y+'_Alpha_'+str(i), x+'_to_'+y+'_Converter_'+str(i))
        G.add_edge(x+'_to_'+y+'_Converter_'+str(i), y)
        return [x+'_to_'+y+'_Alpha_'+str(i), 'g']

    neuron_models = {'ORNs': gen_ORN, 'PNs': gen_PN, 'LNs': gen_LN}
    synapse_models = {'ORNs-LNs': gen_ORNLN_syn, 'LNs-PNs': gen_regsyn_PN, 'PNs-LNs': gen_regsyn, 'ORNs-PNs': gen_ORNPN_syn, 'LNs-LNs': gen_regsyn_LN2}
    interaction_models = {'LNs-ORNs-PNs': ORN_PN_LN_ORN_interaction, 'LNs-ORNs-LNs': ORN_PN_LN_ORN_interaction}
    return neuron_models, synapse_models, interaction_models

In [ ]:
def interaction_weights_default(x,y):
    eps = 1e-6
    A = pairwise_distances(x, y)
    B = 1. / (A+eps)
    return B

def interaction_weights_simple(x,y):
    eps = 1e-6
    A = pairwise_distances(x, y)
    B = 1. / (A+eps)
    return B * 0. + 1.

def interaction_weights_alt(x,y):
    eps = 1e-6
    A = pairwise_distances(x, y)
    B = 1. / (A+eps)
    return B

def interaction_filter_default(B,x,y):
    return B[x,y]>=0.5

def interaction_activation_default(x):
    return x

def simple_circuit_scaler(synapses_a, interaction_group, x, y):
    return synapses_a[x][y].shape[0]

def spiking_circuit(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, 
                    interaction_weights = interaction_weights_default, interaction_filter = interaction_filter_default, interaction_activation = interaction_activation_default, name='default'):
    G = nx.MultiDiGraph()
    visual_neurons = {}
    for cell_group in cell_groups:
        if cell_group in neuron_models:
            for i in cell_groups[cell_group]:
                neuron_models[cell_group](G, i)
                if i in n_data_dict:
                    visual_neurons[i] = n_data_dict[i]

    visual_components = []

    for cell_group_a in cell_groups:
        for cell_group_b in cell_groups:
            synapse_group = cell_group_a + '-' + cell_group_b
            if synapse_group in synapse_groups:
                synapses = synapse_groups[synapse_group]
                if synapse_group in synapse_models:
                    for i in cell_groups[cell_group_a]:
                        for j in cell_groups[cell_group_b]:
                            if i in synapses:
                                if j in synapses[i]:
                                    for k in range(synapses[i][j].shape[0]):
                                        visual_component = synapse_models[synapse_group](G, i, j, k)
                                        if visual_component is not None:
                                            visual_component.append(synapses[i][j][k])
                                            visual_component.append(synapse_group)
                                            visual_components.append(visual_component)

    n_interactions = 0
    interaction_numbers = {}
    affected_units = {}
    for cell_group_a in cell_groups:
        for cell_group_b in cell_groups:
            for cell_group_c in cell_groups:
                interaction_group = cell_group_a + '-' + cell_group_b + '-' + cell_group_c
                interaction_numbers[interaction_group] = {}
                affected_units[interaction_group] = {'0': {}, '1': {}}
                if interaction_group in interaction_models:
                    synapses_a = synapse_groups[cell_group_a + '-' + cell_group_b]
                    synapses_b = synapse_groups[cell_group_b + '-' + cell_group_c]
                    for x in cell_groups[cell_group_a]:
                        interaction_numbers[interaction_group][x] = {}
                        for y in cell_groups[cell_group_b]:
                            interaction_numbers[interaction_group][x][y] = {'kept': 0, 'removed': 0}
                            for z in cell_groups[cell_group_c]:
                                # n_interactions = 0
                                if y in synapses_a[x] and z in synapses_b[y]:
                                    if y not in affected_units[interaction_group]['1']:
                                        affected_units[interaction_group]['1'][y] = {}
                                    if z not in affected_units[interaction_group]['1'][y]:
                                        affected_units[interaction_group]['1'][y][z] = np.zeros((synapses_b[y][z].shape[0],))
                                    if x not in affected_units[interaction_group]['0']:
                                        affected_units[interaction_group]['0'][x] = {}
                                    if y not in affected_units[interaction_group]['0'][x]:
                                        affected_units[interaction_group]['0'][x][y] = np.zeros((synapses_a[x][y].shape[0],))
                                    B = interaction_weights(synapses_a[x][y], synapses_b[y][z])
                                    for u in range(synapses_a[x][y].shape[0]):
                                        for v in range(synapses_b[y][z].shape[0]):
                                            if interaction_filter(B,u,v):
                                                visual_component = interaction_models[interaction_group](G, x, y, z, u, v, interaction_activation(B[u,v]))
                                                if visual_component is not None:
                                                    visual_component.append((synapses_a[x][y][u]+synapses_b[y][z][v])/2)
                                                    visual_component.append(interaction_group)
                                                    visual_components.append(visual_component)
                                                n_interactions += 1
                                                interaction_numbers[interaction_group][x][y]['kept'] += 1
                                                affected_units[interaction_group]['1'][y][z][v] = 1
                                                affected_units[interaction_group]['0'][x][y][u] = 1
                                            else:
                                                interaction_numbers[interaction_group][x][y]['removed'] += 1
                                                
                                            
    print('#Nodes:', len(G.nodes()))
    print('#LNs:', len(LN_uniques))
    print('#Interactions:', n_interactions)
    for i,v in G.nodes(data=True):
        if 'class' not in v:
            print('Error:',i)
    nx.write_gexf(G, '{}.gexf'.format(name))
    np.save('visual_components_{}'.format(name), visual_components)
    np.save('visual_neurons_{}'.format(name), visual_neurons)
    return interaction_numbers, affected_units
    
    

def spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, 
                           interaction_weights = interaction_weights_default, 
                           interaction_filter = interaction_filter_default, 
                           interaction_activation = interaction_activation_default, 
                           name='default', circuit_scaler=simple_circuit_scaler, syn_filter = 10):
    G = nx.MultiDiGraph()
    visual_neurons = {}
    for cell_group in cell_groups:
        if cell_group in neuron_models:
            for i in cell_groups[cell_group]:
                neuron_models[cell_group](G, i)
                if i in n_data_dict:
                    visual_neurons[i] = n_data_dict[i]

    visual_components = []

    for cell_group_a in cell_groups:
        for cell_group_b in cell_groups:
            synapse_group = cell_group_a + '-' + cell_group_b
            if synapse_group in synapse_groups:
                synapses = synapse_groups[synapse_group]
                if synapse_group in synapse_models:
                    for i in cell_groups[cell_group_a]:
                        for j in cell_groups[cell_group_b]:
                            if i in synapses:
                                if j in synapses[i]:
                                    # for k in range(synapses[i][j].shape[0]):
                                    if synapses[i][j].shape[0]>=syn_filter:
                                        visual_component = synapse_models[synapse_group](G, i, j, 0, gain=synapses[i][j].shape[0])
                                        if visual_component is not None:
                                            visual_component.append(synapses[i][j][0])
                                            visual_component.append(synapse_group)
                                            visual_components.append(visual_component)

    n_interactions = 0
    for cell_group_a in cell_groups:
        for cell_group_b in cell_groups:
            for cell_group_c in cell_groups:
                interaction_group = cell_group_a + '-' + cell_group_b + '-' + cell_group_c
                if interaction_group in interaction_models:
                    synapses_a = synapse_groups[cell_group_a + '-' + cell_group_b]
                    synapses_b = synapse_groups[cell_group_b + '-' + cell_group_c]
                    for x in cell_groups[cell_group_a]:
                        for y in cell_groups[cell_group_b]:
                            for z in cell_groups[cell_group_c]:
                                # n_interactions = 0
                                if y in synapses_a[x] and z in synapses_b[y]:
                                    B = interaction_weights(synapses_a[x][y], synapses_b[y][z])
                                    BB = B.copy()
                                    if synapses_a[x][y].shape[0]>=syn_filter and synapses_b[y][z].shape[0]>=syn_filter:
                                        for u in range(1):
                                            for v in range(1):
                                                visual_component = interaction_models[interaction_group](G, x, y, z, u, v, circuit_scaler(synapses_a, interaction_group, x, y) * synapses_b[y][z].shape[0] )
                                                n_interactions += 1


    print('#Nodes:', len(G.nodes()))
    print('#LNs:', len(LN_uniques))
    print('#Interactions:', n_interactions)
    for i,v in G.nodes(data=True):
        if 'class' not in v:
            print('Error:',i)
    nx.write_gexf(G, '{}.gexf'.format(name))
    np.save('visual_components_{}'.format(name), visual_components)
    np.save('visual_neurons_{}'.format(name), visual_neurons)

In [ ]:
def generate_simplest_synapse_groups():
    name_overrides = {}
    LN_ORN_pos_fake = {}
    for i in LN_ORN_pos.keys():
        LN_ORN_pos_fake[i] = {}
        n_syns_fake = {}
        for glom in override_gloms:
            n_syns_fake[glom] = 0
            for j in LN_ORN_pos[i].keys():
                if glom in j and j in cell_groups['ORNs']:
                    n_syns_fake[glom] += LN_ORN_pos[i][j].shape[0]
            n_ORNs = len([i for i in ORN_uniques if glom in i])
            n_av = int(np.round(n_syns_fake[glom]/n_ORNs))

            if n_av>0:
                print(n_av, i, glom)
                for j in [i for i in ORN_uniques if glom in i]:
                    LN_ORN_pos_fake[i][j] = np.zeros((n_av,3))

    synapse_groups = {'ORNs-PNs': ORN_PN_pos, 'ORNs-LNs': ORN_LN_pos, 'LNs-PNs': LN_PN_pos, 'PNs-LNs': PN_LN_pos, 'LNs-ORNs': LN_ORN_pos_fake}
    return synapse_groups

## Compile Circuits for DL5

In [ ]:
nov_ORN_uniques = [i for i in ORN_uniques if 'DL5' in i]
nov_PN_uniques = [i for i in PN_uniques if 'DL5' in i]
nov_LN_uniques = ['LN_1', 'LN_2', 'LN_3']
nov_PN_LN_pos = {}
nov_LN_LN_pos = {}
nov_LN_PN_pos = {}

nov_ORN_LN_pos = {}
nov_LN_ORN_pos = {}

for i in nov_LN_uniques:
    nov_LN_ORN_pos[i] = {}
    nov_LN_LN_pos[i] = {}
    
for i in ['LN_3']:
    for j in ['LN_1', 'LN_2']:
        nov_LN_LN_pos[i][j] = np.zeros((100,3))
        # nov_LN_LN_pos[j][i] = np.zeros((100,3))
    
for i in nov_ORN_uniques:
    nov_ORN_LN_pos[i] = {}
    for j in ['LN_1']:
        nov_ORN_LN_pos[i][j] = np.zeros((100,3))
        
for i in ['LN_1']:
    nov_LN_ORN_pos[i] = {}
    for j in nov_ORN_uniques:
        nov_LN_ORN_pos[i][j] = np.zeros((100,3))
        
for i in nov_PN_uniques:
    nov_PN_LN_pos[i] = {}
    for j in ['LN_2']:
        nov_PN_LN_pos[i][j] = np.zeros((100,3))
        
for i in ['LN_2']:
    nov_LN_PN_pos[i] = {}
    for j in nov_PN_uniques:
        nov_LN_PN_pos[i][j] = np.zeros((100,3))

cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": nov_LN_uniques}
dummy_ORN_LN = {}
dummy_LN_ORN = {}
for i in nov_LN_uniques:
    dummy_LN_ORN[i] = {}
for i in nov_ORN_uniques:
    dummy_ORN_LN[i] = {}
synapse_groups = {'ORNs-PNs': ORN_PN_pos, 'ORNs-LNs': nov_ORN_LN_pos, 'LNs-PNs': nov_LN_PN_pos, 'PNs-LNs': nov_PN_LN_pos, 'LNs-ORNs': nov_LN_ORN_pos, 'LNs-LNs': nov_LN_LN_pos}
neuron_models, synapse_models, interaction_models = setup_spiking_default(ORN_LN_gain=10., ORN_PN_gain=25., LN_PN_gain=2., PN_LN_gain=40., interaction_gain=2e-3, LN_LN_gain=20., exLN_LN_gain=0.)
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DL5_3_all', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": ['LN_1']}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DL5_3_ln1', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": ['LN_1','LN_2']}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DL5_3_ln12', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": ['LN_2']}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DL5_3_ln2', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": []}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DL5_3_lnless', syn_filter=5)

# 2021 End DL5 Inh

nov_ORN_uniques = [i for i in ORN_uniques if 'DL5' in i]
nov_PN_uniques = [i for i in PN_uniques if 'DL5' in i]
nov_LN_uniques = ['LN_1', 'LN_2', 'LN_3']
nov_PN_LN_pos = {}
nov_LN_LN_pos = {}
nov_LN_PN_pos = {}

nov_ORN_LN_pos = {}
nov_LN_ORN_pos = {}

for i in nov_LN_uniques:
    nov_LN_ORN_pos[i] = {}
    nov_LN_LN_pos[i] = {}
    
for i in ['LN_3']:
    for j in nov_LN_uniques:
        nov_LN_LN_pos[i][j] = np.zeros((100,3))
        # nov_LN_LN_pos[j][i] = np.zeros((100,3))
    
for i in nov_ORN_uniques:
    nov_ORN_LN_pos[i] = {}
    for j in ['LN_1']:
        nov_ORN_LN_pos[i][j] = np.zeros((100,3))
        
for i in ['LN_1']:
    nov_LN_ORN_pos[i] = {}
    for j in nov_ORN_uniques:
        nov_LN_ORN_pos[i][j] = np.zeros((100,3))
        
for i in nov_PN_uniques:
    nov_PN_LN_pos[i] = {}
    for j in ['LN_2']:
        nov_PN_LN_pos[i][j] = np.zeros((100,3))
        
for i in ['LN_2']:
    nov_LN_PN_pos[i] = {}
    for j in nov_PN_uniques:
        nov_LN_PN_pos[i][j] = np.zeros((100,3))

cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": nov_LN_uniques}
dummy_ORN_LN = {}
dummy_LN_ORN = {}
for i in nov_LN_uniques:
    dummy_LN_ORN[i] = {}
for i in nov_ORN_uniques:
    dummy_ORN_LN[i] = {}
synapse_groups = {'ORNs-PNs': ORN_PN_pos, 'ORNs-LNs': nov_ORN_LN_pos, 'LNs-PNs': nov_LN_PN_pos, 'PNs-LNs': nov_PN_LN_pos, 'LNs-ORNs': nov_LN_ORN_pos, 'LNs-LNs': nov_LN_LN_pos}
neuron_models, synapse_models, interaction_models = setup_spiking_default(ORN_LN_gain=10., ORN_PN_gain=25., LN_PN_gain=-1., PN_LN_gain=40., interaction_gain=2e-3, LN_LN_gain=20., exLN_LN_gain=0.)
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DL5_3i_all', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": ['LN_1']}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DL5_3i_ln1', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": ['LN_1','LN_2']}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DL5_3i_ln12', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": ['LN_2']}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DL5_3i_ln2', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": []}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DL5_3i_lnless', syn_filter=5)

## Compile Circuits for DM4

In [ ]:
nov_ORN_uniques = [i for i in ORN_uniques if 'DM4' in i]
nov_PN_uniques = [i for i in PN_uniques if 'DM4' in i and 'ad' in i]
nov_LN_uniques = ['LN_1', 'LN_2', 'LN_3']
nov_PN_LN_pos = {}
nov_LN_LN_pos = {}
nov_LN_PN_pos = {}

nov_ORN_LN_pos = {}
nov_LN_ORN_pos = {}

for i in nov_LN_uniques:
    nov_LN_ORN_pos[i] = {}
    nov_LN_LN_pos[i] = {}
    
for i in ['LN_3']:
    for j in ['LN_1', 'LN_2']:
        nov_LN_LN_pos[i][j] = np.zeros((100,3))
        # nov_LN_LN_pos[j][i] = np.zeros((100,3))
    
for i in nov_ORN_uniques:
    nov_ORN_LN_pos[i] = {}
    for j in ['LN_1']:
        nov_ORN_LN_pos[i][j] = np.zeros((100,3))
        
for i in ['LN_1']:
    nov_LN_ORN_pos[i] = {}
    for j in nov_ORN_uniques:
        nov_LN_ORN_pos[i][j] = np.zeros((100,3))
        
for i in nov_PN_uniques:
    nov_PN_LN_pos[i] = {}
    for j in ['LN_2']:
        nov_PN_LN_pos[i][j] = np.zeros((100,3))
        
for i in ['LN_2']:
    nov_LN_PN_pos[i] = {}
    for j in nov_PN_uniques:
        nov_LN_PN_pos[i][j] = np.zeros((100,3))

cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": nov_LN_uniques}
dummy_ORN_LN = {}
dummy_LN_ORN = {}
for i in nov_LN_uniques:
    dummy_LN_ORN[i] = {}
for i in nov_ORN_uniques:
    dummy_ORN_LN[i] = {}
synapse_groups = {'ORNs-PNs': ORN_PN_pos, 'ORNs-LNs': nov_ORN_LN_pos, 'LNs-PNs': nov_LN_PN_pos, 'PNs-LNs': nov_PN_LN_pos, 'LNs-ORNs': nov_LN_ORN_pos, 'LNs-LNs': nov_LN_LN_pos}
neuron_models, synapse_models, interaction_models = setup_spiking_default(ORN_LN_gain=10., ORN_PN_gain=25., LN_PN_gain=2., PN_LN_gain=40., interaction_gain=2e-3, LN_LN_gain=20., exLN_LN_gain=0.)
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4_3_all', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": ['LN_1']}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4_3_ln1', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": ['LN_1','LN_2']}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4_3_ln12', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": ['LN_2']}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4_3_ln2', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": []}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4_3_lnless', syn_filter=5)

nov_ORN_uniques = [i for i in ORN_uniques if 'DM4' in i]
nov_PN_uniques = [i for i in PN_uniques if 'DM4' in i and 'ad' in i]
nov_LN_uniques = ['LN_1', 'LN_2', 'LN_3']
nov_PN_LN_pos = {}
nov_LN_LN_pos = {}
nov_LN_PN_pos = {}

nov_ORN_LN_pos = {}
nov_LN_ORN_pos = {}

for i in nov_LN_uniques:
    nov_LN_ORN_pos[i] = {}
    nov_LN_LN_pos[i] = {}
    
for i in ['LN_3']:
    for j in nov_LN_uniques:
        nov_LN_LN_pos[i][j] = np.zeros((100,3))
        # nov_LN_LN_pos[j][i] = np.zeros((100,3))
    
for i in nov_ORN_uniques:
    nov_ORN_LN_pos[i] = {}
    for j in ['LN_1']:
        nov_ORN_LN_pos[i][j] = np.zeros((100,3))
        
for i in ['LN_1']:
    nov_LN_ORN_pos[i] = {}
    for j in nov_ORN_uniques:
        nov_LN_ORN_pos[i][j] = np.zeros((100,3))
        
for i in nov_PN_uniques:
    nov_PN_LN_pos[i] = {}
    for j in ['LN_2']:
        nov_PN_LN_pos[i][j] = np.zeros((100,3))
        
for i in ['LN_2']:
    nov_LN_PN_pos[i] = {}
    for j in nov_PN_uniques:
        nov_LN_PN_pos[i][j] = np.zeros((100,3))

cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": nov_LN_uniques}
dummy_ORN_LN = {}
dummy_LN_ORN = {}
for i in nov_LN_uniques:
    dummy_LN_ORN[i] = {}
for i in nov_ORN_uniques:
    dummy_ORN_LN[i] = {}
synapse_groups = {'ORNs-PNs': ORN_PN_pos, 'ORNs-LNs': nov_ORN_LN_pos, 'LNs-PNs': nov_LN_PN_pos, 'PNs-LNs': nov_PN_LN_pos, 'LNs-ORNs': nov_LN_ORN_pos, 'LNs-LNs': nov_LN_LN_pos}
neuron_models, synapse_models, interaction_models = setup_spiking_default(ORN_LN_gain=10., ORN_PN_gain=25., LN_PN_gain=-1., PN_LN_gain=40., interaction_gain=2e-3, LN_LN_gain=20., exLN_LN_gain=0.)
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4_3i_all', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": ['LN_1']}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4_3i_ln1', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": ['LN_1','LN_2']}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4_3i_ln12', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": ['LN_2']}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4_3i_ln2', syn_filter=5)
cell_groups = {'ORNs': nov_ORN_uniques, 
               'PNs': nov_PN_uniques, 
               "LNs": []}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4_3i_lnless', syn_filter=5)

## Compile a Circuit with Interconnected Glomeruli

In [ ]:
nov_ORN_uniques = [i for i in ORN_uniques if 'DM4' in i]
nov_PN_uniques = [i for i in PN_uniques if 'DM4' in i and 'ad' in i]
nov_ORNb_uniques = [i for i in ORN_uniques if 'DL5' in i]
nov_PNb_uniques = [i for i in PN_uniques if 'DL5' in i]
nov_LN_uniques = ['LN_1', 'LN_2', 'LN_1b', 'LN_2b', 'LN_3']
nov_PN_LN_pos = {}
nov_LN_LN_pos = {}
nov_LN_PN_pos = {}

nov_ORN_LN_pos = {}
nov_LN_ORN_pos = {}

for i in nov_LN_uniques:
    nov_LN_ORN_pos[i] = {}
    nov_LN_LN_pos[i] = {}
    
for i in ['LN_3']:
    for j in ['LN_1', 'LN_2', 'LN_1b', 'LN_2b']:
        nov_LN_LN_pos[i][j] = np.zeros((5000,3))
        nov_LN_LN_pos[j][i] = np.zeros((100,3))
    
# DM4 ORNs
for i in nov_ORN_uniques:
    nov_ORN_LN_pos[i] = {}
    for j in ['LN_1']:
        nov_ORN_LN_pos[i][j] = np.zeros((100,3))
        
for i in ['LN_1']:
    nov_LN_ORN_pos[i] = {}
    for j in nov_ORN_uniques:
        nov_LN_ORN_pos[i][j] = np.zeros((100,3))
        
# DL5 ORNs
for i in nov_ORNb_uniques:
    nov_ORN_LN_pos[i] = {}
    for j in ['LN_1b']:
        nov_ORN_LN_pos[i][j] = np.zeros((100,3))
        
for i in ['LN_1b']:
    nov_LN_ORN_pos[i] = {}
    for j in nov_ORNb_uniques:
        nov_LN_ORN_pos[i][j] = np.zeros((100,3))

# DM4 PNs
for i in nov_PN_uniques:
    nov_PN_LN_pos[i] = {}
    for j in ['LN_2']:
        nov_PN_LN_pos[i][j] = np.zeros((100,3))
        
for i in ['LN_2']:
    nov_LN_PN_pos[i] = {}
    for j in nov_PN_uniques:
        nov_LN_PN_pos[i][j] = np.zeros((100,3))
        
# DL5 PNs
for i in nov_PNb_uniques:
    nov_PN_LN_pos[i] = {}
    for j in ['LN_2b']:
        nov_PN_LN_pos[i][j] = np.zeros((100,3))
        
for i in ['LN_2b']:
    nov_LN_PN_pos[i] = {}
    for j in nov_PNb_uniques:
        nov_LN_PN_pos[i][j] = np.zeros((100,3))

cell_groups = {'ORNs': nov_ORN_uniques + nov_ORNb_uniques, 
               'PNs': nov_PN_uniques + nov_PNb_uniques, 
               "LNs": ['LN_1', 'LN_2', 'LN_1b', 'LN_2b'], 'exLNs': ['LN_3']}
dummy_ORN_LN = {}
dummy_LN_ORN = {}
for i in nov_LN_uniques:
    dummy_LN_ORN[i] = {}
for i in nov_ORN_uniques:
    dummy_ORN_LN[i] = {}
synapse_groups = {'ORNs-PNs': ORN_PN_pos, 'ORNs-LNs': nov_ORN_LN_pos, 'LNs-PNs': nov_LN_PN_pos, 'PNs-LNs': nov_PN_LN_pos, 'LNs-ORNs': nov_LN_ORN_pos, 'LNs-LNs': nov_LN_LN_pos,
                  'LNs-exLNs': nov_LN_LN_pos, 'exLNs-LNs': nov_LN_LN_pos}
neuron_models, synapse_models, interaction_models = setup_spiking_default(ORN_LN_gain=10., ORN_PN_gain=25., LN_PN_gain=2., PN_LN_gain=40., interaction_gain=2e-3, LN_LN_gain=100., exLN_LN_gain=200., LN_exLN_gain=200.)

spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4DL5_3_full', syn_filter=5)
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4DL5_3_full_2', syn_filter=5)

cell_groups = {'ORNs': nov_ORN_uniques + nov_ORNb_uniques, 
               'PNs': nov_PN_uniques + nov_PNb_uniques, 
               "LNs": ['LN_1', 'LN_1b'], 'exLNs': []}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4DL5_3_ln1', syn_filter=5)

cell_groups = {'ORNs': nov_ORN_uniques + nov_ORNb_uniques, 
               'PNs': nov_PN_uniques + nov_PNb_uniques, 
               "LNs": ['LN_2', 'LN_2b'], 'exLNs': []}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4DL5_3_ln2', syn_filter=5)

cell_groups = {'ORNs': nov_ORN_uniques + nov_ORNb_uniques, 
               'PNs': nov_PN_uniques + nov_PNb_uniques, 
               "LNs": ['LN_1', 'LN_1b', 'LN_2', 'LN_2b'], 'exLNs': []}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4DL5_3_ln12', syn_filter=5)

cell_groups = {'ORNs': nov_ORN_uniques + nov_ORNb_uniques, 
               'PNs': nov_PN_uniques + nov_PNb_uniques, 
               "LNs": [], 'exLNs': []}
spiking_circuit_simple(cell_groups, synapse_groups, neuron_models, synapse_models, interaction_models, name='DM4DL5_3_lnless', syn_filter=5)
